In [57]:
import sympy as sy
import numpy as np
from sympy.physics.vector import ReferenceFrame,dynamicsymbols,Point,express
from sympy.physics.mechanics import inertia
from sympy.physics.vector.printing import vlatex

import IPython
from IPython.display import display

def displayH(a1,a2='', a3='', a4='', a5='', a6='', a7='',):
    latex_a1 = sy.latex(a1)
    latex_a2 = sy.latex(a2)
    latex_a3 = sy.latex(a3)
    latex_a4 = sy.latex(a4)
    latex_a5 = sy.latex(a5)
    latex_a6 = sy.latex(a6)
    latex_a7 = sy.latex(a7)
    display( IPython.core.display.Math(latex_a1 + latex_a2 + latex_a3 + latex_a4 + latex_a5 + latex_a6 + latex_a7)  )

In [116]:
ECI_Frame = ReferenceFrame('N')

w_e = sy.Symbol("w_e")
t = sy.Symbol("t")

w_e = 7.2921159e-5
t = 86400

gamma = w_e*t

ECEF_Frame = ECI_Frame.orientnew('E','Axis',[gamma,ECI_Frame.z])

RAAN = dynamicsymbols("Omega")
INC = dynamicsymbols("i")
AOP = dynamicsymbols("omega")
F = dynamicsymbols("theta")

RAAN = 15/180*np.pi
INC = 75/180*np.pi
AOP = 45/180*np.pi
F = 15/180*np.pi

Perifocal_Frame = ECI_Frame.orientnew('O','Body',(RAAN,INC,AOP+F),'313')

Earth_Center = Point('O')
Earth_Center.set_vel(ECI_Frame,0)
Earth_Center.set_vel(ECEF_Frame,0)
Earth_Center.set_vel(Perifocal_Frame,0)

psi = dynamicsymbols("psi")
theta = dynamicsymbols("theta")
phi = dynamicsymbols("phi")

psi = 1/180*np.pi
theta = 1/180*np.pi
phi = 1/180*np.pi

Body_Frame = Perifocal_Frame.orientnew('B','Body',(psi,theta,phi),'321')

R = sy.Symbol("R")
R = 6378+600
V = sy.Symbol("V")
V = 7.61268398902
Body_Center = Earth_Center.locatenew("R_B",R*Perifocal_Frame.x)
Body_Center.set_vel(Body_Frame,0)
Body_Center.set_vel(Perifocal_Frame,V*Perifocal_Frame.y)

G = sy.Symbol("G")
M = sy.Symbol("M")

G = (6.6743*1e-11)/1e9
M = 5.972e24

n = sy.Symbol("n")
n = sy.sqrt(G*M/R**3)

Perifocal_Frame.set_ang_vel(ECI_Frame,n*Perifocal_Frame.y)

In [117]:
R_Body = Body_Center.pos_from(Earth_Center).express(Body_Frame)

I1 = sy.Symbol("I_1")
I2 = sy.Symbol("I_2")
I3 = sy.Symbol("I_3")

I1 = 400
I2 = 300
I3 = 200

I_Body = inertia(ixx=I1,iyy=I2,izz=I3,frame=Body_Frame)

Lg_Body = 3*G*M/R**5*(R_Body.cross(I_Body.dot(R_Body)))

displayH(sy.Symbol(vlatex(Lg_Body.to_matrix(Body_Frame))))

I_Perifocal = I_Body.express(Perifocal_Frame)

R_Perifocal = Body_Center.pos_from(Earth_Center).express(Perifocal_Frame)

Lg_O = 3*G*M/R**5*(R_Perifocal.cross(I_Perifocal.dot(R_Perifocal)))

displayH(sy.Symbol(vlatex(Lg_O.to_matrix(Perifocal_Frame))))

I_ECI = I_Body.express(ECI_Frame)

R_ECI = Body_Center.pos_from(Earth_Center).express(ECI_Frame)

Lg_ECI = 3*G*M/R**5*(R_ECI.cross(I_ECI.dot(R_ECI)))

displayH(sy.Symbol(vlatex(Lg_ECI.to_matrix(ECI_Frame))))

I_ECEF = I_Body.express(ECEF_Frame)

R_ECEF = Body_Center.pos_from(Earth_Center).express(ECEF_Frame)

Lg_ECEF = 3*G*M/R**5*(R_ECEF.cross(I_ECEF.dot(R_ECEF)))

displayH(sy.Symbol(vlatex(Lg_ECEF.to_matrix(ECEF_Frame))))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [122]:
for frame in [Body_Frame,Perifocal_Frame,ECI_Frame,ECEF_Frame]:
    V = Body_Center.vel(frame)
    displayH(V.to_matrix(frame),"->",np.linalg.norm(np.array(V.to_matrix(frame),dtype=float)))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>